# "Boring Lots": An Analysis of Parking Lot Properties along The Boring Company's proposed DC-Baltimore Route

By Russell Tran using publicly available data (not affiliated at the time)


In [1]:
import urllib.request
import json
import csv
import time
import datetime
import pandas as pd

## Prologue: Practice with the Google Maps API

Credits to [ritvikmath](https://github.com/ritvikmath/ScrapingData/blob/master/Scraping%20GMaps%20Data.ipynb) for this practice code

In [2]:
"""
INPUTS:
    url: a request url
OUTPUTS: 
    the data returned by calling that url
"""
def request_data_from_url(url):
    req = urllib.request.Request(url)
    success = False
    while success is False:
        try: 
            #open the url
            response = urllib.request.urlopen(req)
            
            #200 is the success code for http
            if response.getcode() == 200:
                success = True
        except Exception as e:
            #if we didn't get a success, then print the error and wait 5 seconds before trying again
            print (e)
            time.sleep(5)

            print ("Error for URL %s: %s" % (url, datetime.datetime.now()))
            print ("Retrying...")

    #return the contents of the response
    return response.read()

In [3]:
"""
INPUTS:
    api_key: authentication to GMaps that we're allowed to request this data
    origin: lat,long of origin
    destination: lat,long of destination
    frequency: how often to scrape the data
    duration: how long to scrape the data
OUTPUTS:
    nothing, simply continues to write data to spreadsheet
"""
def scrape_gmaps_data(api_key, origin, destination, frequency, duration):
    
    #we want to scrape the googlemaps website
    site = 'https://maps.googleapis.com/maps/api/'
    
    #we want to use the distancematrix service
    service = 'distancematrix/json?'
    
    #input origin and destination from the user 
    locations = 'origins=%s&destinations=%s&departure_time=now&' % (origin, destination)
    
    #input api key from user
    key = 'key=%s' % (api_key)
    
    #construct request url
    request_url = site + service + locations + key
    

    
    #create dataframe, write the header row
    df = pd.DataFrame(["timestamp", "travel_time"])
            
    #get the travel time at regular intervals
    step = 1
    while (step <= int(duration*60 / frequency)):
        #convert response to python dictionary 
        data = json.loads(request_data_from_url(request_url))
            
        #extract travel time from response
        traffic_time = data['rows'][0]['elements'][0]['duration_in_traffic']['value']
            
        #write to our dataframe
        df.append([datetime.datetime.now(), traffic_time])
        
        #iterate
        if step % 10 == 0:
                print (str(step) + ' datapoints gathered ...')
            
        step += 1
        time.sleep(frequency*60)
        
    #save dataframe as csv
    df.to_csv("ucla_to_ucsd.csv")

In [ ]:
api_key = input('Please Enter Your API Key: ')
origin = '34.070243,-118.436293' #UCLA
destination = '32.881266,-117.233290' #UCSD
frequency = int(input('How Often to Scrape Data (minutes): '))
duration = int(input('How Long to Scrape Data (hours): '))

if __name__ == '__main__':
    scrape_gmaps_data(api_key, origin, destination, frequency, duration)

In [ ]:
api_key = "-----"
origin = '34.070243,-118.436293' #UCLA
destination = '32.881266,-117.233290' #UCSD



#we want to scrape the googlemaps website
site = 'https://maps.googleapis.com/maps/api/'
    
#we want to use the distancematrix service
service = 'distancematrix/json?'
    
#input origin and destination from the user 
locations = 'origins=%s&destinations=%s&departure_time=now&' % (origin, destination)
    
#input api key from user
key = 'key=%s' % (api_key)
    
#construct request url
request_url = site + service + locations + key
    

    
#create dataframe, write the header row
df = pd.DataFrame(columns= ["travel time"], index=["timestamp"])
df.loc["12345"] = ["damn"]
    
    
    
    
    
#convert response to python dictionary 
data = json.loads(request_data_from_url(request_url))
            
#extract travel time from response
traffic_time = data['rows'][0]['elements'][0]['duration_in_traffic']['value']
            
#write to our dataframe
df.loc[datetime.datetime.now()] = [traffic_time]
    
    
    
    
#save dataframe as csv
df.to_csv("stupid.csv")

In [ ]:
print (df)


In [ ]:
print (request_url)

## Querying all parking lots within 500m (~0.3mi) from 53 New York Ave NE

53 New York Ave NE, Washington, D.C. is a parking lot [currently owned by The Boring Company](https://arstechnica.com/cars/2018/02/the-boring-company-gets-a-permit-to-dig-up-washington-dc-parking-lot/) with a hole dug in it

In [ ]:
api_key = "-----"

location = '38.90769,-77.007' # coordinates of 53 New York Ave NE

# we want to scrape the googlemaps website
site = 'https://maps.googleapis.com/maps/api/'
    
# we want to use the place search service
service = 'place/findplacefromtext/json?'

#set parameters
text_input = "parking%lot"
input_type = "textquery"
radius = 500 # 500 meters aka ~0.3 mi
location_bias = "circle:%s@%s" % (radius, location)
parameters = 'input=%s&inputtype=%s&locationbias=%s' % (text_input, input_type, location_bias)
    
#input api key from user
key = '&key=%s' % (api_key)
    
#construct request url
request_url = site + service + parameters + key

print(request_url)

In [ ]:
#convert response to python dictionary 
data = json.loads(request_data_from_url(request_url))

data

In [ ]:
request_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=38.90769,-77.007&radius=500&keyword=parking%lot&key=-----"

print(request_url)

In [ ]:
#convert response to python dictionary 
data = json.loads(request_data_from_url(request_url))

latitudes = []
longitudes = []

#extract lat & lng from response
for entry in data['results']:
    
    lat_result = entry['geometry']['location']['lat']
    lng_result = entry['geometry']['location']['lng']
    
    latitudes.append(lat_result)
    longitudes.append(lng_result)
    
print(latitudes)
print(longitudes)

In [ ]:
# convert to array of tuples
marker_locations = []

for i in range(len(latitudes)):
    coordinate = (latitudes[i], longitudes[i])
    marker_locations.append(coordinate)
    
marker_locations

In [ ]:
import gmaps
import gmaps.datasets

gmaps.configure(api_key="-----")

fig = gmaps.figure(map_type='SATELLITE')
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

## Conversion of GPS coordinates to addresses

In [ ]:
place_ids = []

#extract place_id from response
for entry in data['results']:
    
    place_id_result = entry['place_id']
    
    place_ids.append(place_id_result)
    
place_ids

In [ ]:
# Place Details Google Maps API Call for each of these found parking lots

fields = "address_component,adr_address,formatted_address,geometry,id,name,permanently_closed,photo,place_id,plus_code,type,url,vicinity,formatted_phone_number,opening_hours,website,price_level,rating,review"

parking_lot_details = pd.Series()


for place_id in place_ids: 
    
    request_url = "https://maps.googleapis.com/maps/api/place/details/json?placeid=%s&fields=%s&key=-----" % (place_id, fields)
    detailed_data = json.loads(request_data_from_url(request_url))
    
    # store each JSON file (aka dictionary) in the Pandas Series
    # mapped to each place_id
    parking_lot_details[place_id] = detailed_data
    
parking_lot_details

In [ ]:
example_request_url = "https://maps.googleapis.com/maps/api/place/details/json?placeid=%s&fields=%s&key=-----" % ('ChIJaTmI6xu4t4kRN5zaPe-TeDc', fields)

print(example_request_url)

In [ ]:
formatted_addresses = []

#extract formattes_addresses from response
for detailed_data in parking_lot_details:
    
        formatted_address_result = detailed_data['result']['formatted_address']    
        formatted_addresses.append(formatted_address_result)
    
formatted_addresses

### How feasibly can these addresses be found on Reonomy (how well do Google Maps addresses overlap with Reonomy addresses)? And how accurate are the Google Maps Queries at finding parking lots?

Manually entered into Reonomy : noted outcome

 '100 Florida Ave NE, Washington, DC 20002, USA', : Large building likely containing a parking structure 
 
 '100L N St NE, Washington, DC 20002, USA', : **not found, but top result is 33 New York Ave NE, Washington, DC 20002, which is the Hyatt Hotel**
 
 '1100 First St NE, Washington, DC 20001, USA', returned a big office building
 
 '1205 3rd St NE, Washington, DC 20002, USA', : **not found, returned 1215 instead, which is a large parking lot though!**
 
 '3218, 1275 First St NE, Washington, DC 20002, USA',  : **not found on Reonomy**
 
 '64 New York Ave NE, Washington, DC 20002, USA', : successfully reutnred a parking lot / "Commercial Vacant Land"
 
 '49O N St NE, Washington, DC 20002, USA', **not found, top result is 75 P St NE, Washington, DC 20002, an office building**
 
 '77 Patterson St NE, Washington, DC 20002, USA', : **not found, returned 1200 1st St NE, Washington, DC 20002 which is an office building**
 
 '23K St NW, Washington, DC 20001, USA', **not found, but top result is 1010 N Capitol St NW, Washington, DC 20002, which is indeed a parking lot!**
 
 
 TODO:
 
 
 '170L N St NE, Washington, DC 20002, USA',
 
 '1520 Eckington Pl NE, Washington, DC 20002, USA',
 
 '301 Florida Ave NE, Washington, DC 20002, USA',
 
 '30M N St NE, Washington, DC 20002, USA',
 
 '17 M St NE, Washington, DC 20002, USA',
 
 '83N N St NE, Washington, DC 20002, USA',
 
 '55 New York Ave NE, Washington, DC 20002, USA', : returned a genuine parking lot
 
 '53P N St NE, Washington, DC 20002, USA',
 
 '111N N St NE, Washington, DC 20002, USA',
 
 '130 M St NE, Washington, DC 20002, USA',
 
 '175N N St NE, Washington, DC 20002, USA'
 

## How about using Corelogic?

'100 Florida Ave NE, Washington, DC 20002, USA'


 '100L N St NE, Washington, DC 20002, USA'
 '1100 First St NE, Washington, DC 20001, USA'
 '1205 3rd St NE, Washington, DC 20002, USA'
 '3218, 1275 First St NE, Washington, DC 20002, USA'
 '64 New York Ave NE, Washington, DC 20002, USA'
 '49O N St NE, Washington, DC 20002, USA'
 '77 Patterson St NE, Washington, DC 20002, USA'
 '23K St NW, Washington, DC 20001, USA'
 '170L N St NE, Washington, DC 20002, USA'
 '1520 Eckington Pl NE, Washington, DC 20002, USA'
 '301 Florida Ave NE, Washington, DC 20002, USA'
 '30M N St NE, Washington, DC 20002, USA'
 '17 M St NE, Washington, DC 20002, USA'
 '83N N St NE, Washington, DC 20002, USA'
 '55 New York Ave NE, Washington, DC 20002, USA'
 '53P N St NE, Washington, DC 20002, USA'
 '111N N St NE, Washington, DC 20002, USA'
 '130 M St NE, Washington, DC 20002, USA'
 '175N N St NE, Washington, DC 20002, USA'
 


## Attom Data Solutions

In [ ]:
import http.client 


def attom_property_detail_call(address1, address2):
    conn = http.client.HTTPSConnection("search.onboard-apis.com") 

    headers = { 
        'accept': "application/json", 
        'apikey': "-----", 
    } 

    conn.request("GET", "/propertyapi/v1.0.0/property/detail?address1=%s&address2=%s" % (address1, address2), headers=headers) 

    res = conn.getresponse() 
    data = res.read() 
    
    return data.decode("utf-8")

print(attom_property_detail_call("4529%20Winona%20Court", "Denver%2C%20CO"))
print('\n\n')

In [ ]:
sample_addresses = [
    '100 Florida Ave NE, Washington, DC 20002, USA',
     '100L N St NE, Washington, DC 20002, USA',
     '1100 First St NE, Washington, DC 20001, USA',
     '1205 3rd St NE, Washington, DC 20002, USA',
     '3218, 1275 First St NE, Washington, DC 20002, USA',
     '64 New York Ave NE, Washington, DC 20002, USA',
     '49O N St NE, Washington, DC 20002, USA',
     '77 Patterson St NE, Washington, DC 20002, USA',
     '23K St NW, Washington, DC 20001, USA',
     '170L N St NE, Washington, DC 20002, USA',
     '1520 Eckington Pl NE, Washington, DC 20002, USA',
     '301 Florida Ave NE, Washington, DC 20002, USA',
     '30M N St NE, Washington, DC 20002, USA',
     '17 M St NE, Washington, DC 20002, USA',
     '83N N St NE, Washington, DC 20002, USA',
     '55 New York Ave NE, Washington, DC 20002, USA',
     '53P N St NE, Washington, DC 20002, USA',
     '111N N St NE, Washington, DC 20002, USA',
     '130 M St NE, Washington, DC 20002, USA',
     '175N N St NE, Washington, DC 20002, USA'
]



formatted_addresses_split = []
for address in sample_addresses:
    # split the addresses by commas
    line = str.split(address, ',')
    # split the state and zip by the space
    state_and_zip = str.split(line[2], ' ')
    print(state_and_zip)
    # line = [line[0], line[1][1:], state_and_zip[0], state_and_zip[1], line[3]]
    formatted_addresses_split.append(line)
    

formatted_addresses_split

In [ ]:
formatted_addresses_split = [
     ['100 Florida Ave NE', 'Washington, DC', '20002', 'USA'],
     ['100L N St NE', 'Washington, DC', '20002', 'USA'],
     ['1100 First St NE', 'Washington, DC', '20002', 'USA'],
     ['1205 3rd St NE', 'Washington, DC', '20002', 'USA'],
     ['1275 First St NE', 'Washington, DC', '20002', 'USA'],
     ['64 New York Ave NE', 'Washington, DC', '20002', 'USA'],
     ['49O N St NE', 'Washington, DC', '20002', 'USA'],
     ['77 Patterson St NE', 'Washington, DC', '20002', 'USA'],
     ['23K St NW', 'Washington, DC', '20002', 'USA'],
     ['170L N St NE', 'Washington, DC', '20002', 'USA'],
     ['1520 Eckington Pl NE', 'Washington, DC', '20002', 'USA'],
     ['301 Florida Ave NE', 'Washington, DC', '20002', 'USA'],
     ['30M N St NE', 'Washington, DC', '20002', 'USA'],
     ['17 M St NE', 'Washington, DC', '20002', 'USA'],
     ['83N N St NE', 'Washington, DC', '20002', 'USA'],
     ['55 New York Ave NE', 'Washington, DC', '20002', 'USA'],
     ['53P N St NE', 'Washington, DC', '20002', 'USA'],
     ['111N N St NE', 'Washington, DC', '20002', 'USA'],
     ['130 M St NE', 'Washington, DC', '20002', 'USA'],
     ['175N N St NE', 'Washington, DC', '20002', 'USA']
]

formatted_addresses_split

In [ ]:
attom_addresses = []

for line in formatted_addresses_split:
    address1 = (line[0]).replace(' ', '%20')
    address2 = (line[1]).replace(' ', '%20').replace(',', '%2C')
    
    attom_addresses.append([address1, address2])
    
attom_addresses

In [ ]:
for i in range(len(attom_addresses)):
    print(formatted_addresses_split[i])
    print(':\n')
    
    print (attom_property_detail_call(address1=attom_addresses[i][0], address2=attom_addresses[i][1]))
    

12 of the 20 calls had SuccessWithoutResult

### So the address-based lookup was bad, but what about GPS based lookup for the same 20 properties?

In [ ]:


conn = http.client.HTTPSConnection("search.onboard-apis.com") 

headers = { 
    'accept': "application/json", 
    'apikey': "-----", 
} 

stupid_shit = "/propertyapi/v2.0.0/propertysearch/geo?latitude={latitude}&longitude={longitude}&radius={radius}".format(latitude='41.695210',longitude='-87.671460',radius='5')
conn.request("GET", stupid_shit, headers=headers) 

res = conn.getresponse() 
data = res.read() 

print("/propertyapi/v1.0.0/property/detail?address1=4529%20Winona%20Court&address2=Denver%2C%20CO")
print("\n")
print(stupid_shit)
data.decode("utf-8")


In [ ]:
def attom_gps_based_lookup(lat, long):

    conn = http.client.HTTPSConnection("search.onboard-apis.com") 

    headers = { 
        'accept': "application/json", 
        'apikey': "-----", 
    } 

    conn.request("GET", "/propertyapi/v1.0.0/property/v2/propertysearch/geo/41.695210/-87.671460/5" % (address1, address2), headers=headers) 

    res = conn.getresponse() 
    data = res.read() 

    return data.decode("utf-8")

In [ ]:
'''

import http.client 

conn = http.client.HTTPSConnection("search.onboard-apis.com") 

headers = { 
    'accept': "application/json", 
    'apikey': "", 
} 

conn.request("GET", "/propertyapi/v1.0.0/property/detail?address1=4529%20Winona%20Court&address2=Denver%2C%20CO", headers=headers) 

res = conn.getresponse() 
data = res.read() 

print(data.decode("utf-8"))

"https://search.onboard-apis.com/propertyapi/v1.0.0/property/detail?address=468%20SEQUOIA%20DR%2C%20SMYRNA%2C%20DE"

# ===========================================
'''
             
             
# "https://api.attomdata.com/property/v2/propertysearch/geo/41.695210/-87.671460/5"
             
             
import http.client 

conn = http.client.HTTPSConnection("api.attomdata.com") 

headers = { 
    'accept': "application/json", 
    'apikey': "-----", 
} 

conn.request("GET", "/property/v2/propertysearch/geo/41.695210/-87.671460/5", headers=headers) 

res = conn.getresponse() 
data = res.read() 

print(conn)
print(res)
print(data)
print(data.decode("utf-8"))

print(res.msg)
#print('\n')
print(res.status)
#print('\n')
print(res.reason)
